In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

clfs = {
    "CART": DecisionTreeClassifier(random_state=1234, max_depth=1000),
    "RNF": RandomForestClassifier(random_state=1234),
    "XGB": XGBClassifier(use_label_encoder=False),
    "CAT": CatBoostClassifier(random_state=1234),
    "ADA": AdaBoostClassifier(DecisionTreeClassifier(random_state=1234, max_depth=1000)),
    "BAG": BaggingClassifier(DecisionTreeClassifier(random_state=1234, max_depth=1000)),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234),
    "DL": DecisionListClassifier(random_state=1234) 
  
}

dataset = 'campus'

import worstcase_helper
import importlib
importlib.reload(worstcase_helper)

preprocess, X, y = worstcase_helper.load_dataset_with_preprocess(dataset)

from sklearn.pipeline import make_pipeline
def make_pipeline_clf(clf_name):
    clf = make_pipeline(
        preprocess,
        clfs[clf_name]
    )
    return clf

clfs_list = []
for clf_id, clf_name in enumerate(clfs):
    clf = make_pipeline_clf(clf_name)
    clf.fit(X, y)
    clfs_list.append(clf)

import pickle

for clf_id, clf_name in enumerate(clfs):
    pickle.dump(clfs_list[clf_id], open(f"./new_models/campus_{clf_name}.p", 'wb'))


/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


[10:57:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.005344
0:	learn: 0.6886312	total: 68ms	remaining: 1m 7s
1:	learn: 0.6844857	total: 97.6ms	remaining: 48.7s
2:	learn: 0.6807358	total: 103ms	remaining: 34.3s
3:	learn: 0.6768815	total: 108ms	remaining: 26.8s
4:	learn: 0.6726281	total: 115ms	remaining: 22.9s
5:	learn: 0.6692826	total: 119ms	remaining: 19.7s
6:	learn: 0.6646894	total: 123ms	remaining: 17.4s
7:	learn: 0.6603944	total: 126ms	remaining: 15.6s
8:	learn: 0.6561682	total: 131ms	remaining: 14.4s
9:	learn: 0.6528167	total: 152ms	remaining: 15.1s
10:	learn: 0.6483887	total: 158ms	remaining: 14.2s
11:	learn: 0.6453732	total: 164ms	remaining: 13.5s
12:	learn: 0.6409700	total: 173ms	remaining: 13.1s
13:	learn: 0.6376258	total: 181ms	remaining: 12.7s
14:	learn: 0.6

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed 

In [2]:
# clf_cart_exp = dx.Explainer(clf_cart, X, y, label="CART")
import dalex as dx
clfs_explainers = []
for clf_id, clf_name in enumerate(clfs):
    clf_exp = dx.Explainer(clfs_list[clf_id], X, y, label=clf_name)
    clfs_explainers.append(clf_exp)

Preparation of a new explainer is initiated

  -> data              : 215 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 215 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : CART
  -> predict function  : <function yhat_proba_default at 0x7fb228cc0ee0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.688, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = 0.0, mean = 0.0, max = 0.0
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 215 rows 12 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a nump

In [3]:
cat_feat = ['gender', 'ssc_b', 'hsc_b', 'hsc_s',
                                  'degree_t', 'workex', 'specialisation']
cont_feat = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p',
                                  'mba_p']

In [4]:
# import dalex as dx
clfs_pd_cat = []
clfs_pd_cont = []
for clf_id, clf_name in enumerate(clfs):
    clf_pd_cat = clfs_explainers[clf_id].model_profile( variables = cat_feat,
                                                        variable_type='categorical')
    clfs_pd_cat.append(clf_pd_cat)

for clf_id, clf_name in enumerate(clfs):
    clf_pd_cont = clfs_explainers[clf_id].model_profile( variables = cont_feat)
    clfs_pd_cont.append(clf_pd_cont)

Calculating ceteris paribus: 100%|██████████| 5/5 [00:04<00:00,  1.24it/s]


In [5]:
# import dalex as dx
clfs_mp = []
for clf_id, clf_name in enumerate(clfs):
    clf_mp = clfs_explainers[clf_id].model_parts()
    clfs_mp.append(clf_mp)

In [6]:
import numpy as np 
campus_mp_results = np.zeros(shape=len(clfs))

for i in range(len(clfs)):
    campus_mp_results[i] = np.abs(np.subtract(clfs_mp[0].result.dropout_loss[1:-1], clfs_mp[i].result.dropout_loss[1:-1])).sum()

In [7]:
import pandas as pd
pd.DataFrame(campus_mp_results).to_csv(f"./results/mp_campus.csv")

In [8]:
campus_pd_cat_results = np.zeros(shape=len(clfs))
campus_pd_cont_results = np.zeros(shape=len(clfs))
for i in range(len(clfs)):
    campus_pd_cat_results[i] = np.abs(np.subtract(clfs_pd_cat[0].result._yhat_, clfs_pd_cat[i].result._yhat_)).sum()
    campus_pd_cont_results[i] = np.abs(np.subtract(clfs_pd_cont[0].result._yhat_, clfs_pd_cont[i].result._yhat_)).sum()

In [9]:
campus_pd_results = campus_pd_cat_results + campus_pd_cont_results

In [10]:
pd.DataFrame(campus_pd_results).to_csv(f"./results/pd_campus.csv")